<a href="https://colab.research.google.com/github/dizzySummer/NLP-TensorFlow2.0/blob/master/amazonwbpe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '/content/drive/My Drive/amazon-fine-food-reviews/')

import attention


In [0]:
import io
import os
import pandas as pd
import numpy as np
#import data that stored on Google drive
import io
df = pd.read_csv('/content/drive/My Drive/amazon-fine-food-reviews/Reviews.csv')
# Dataset is now stored in a Pandas Dataframe
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [0]:
# this block will be needed in the real dataset
df=df.dropna()
df=df[['Summary','Text']].reset_index(drop=True)
df.head()

len(df) #568411

568411

In [0]:
print("preprocessing")
df["Text"]=df["Text"].str.lower()
df["Summary"]=df["Summary"].str.lower()

df.head()

preprocessing


,Summary,Text
0,good quality dog food,i have bought several of the vitality canned d...
1,not as advertised,product arrived labeled as jumbo salted peanut...
2,"""delight"" says it all",this is a confection that has been around a fe...
3,cough medicine,if you are looking for the secret ingredient i...
4,great taffy,great taffy at a great price. there was a wid...


In [0]:
!pip install sentencepiece

     |████████████████████████████████| 1.0MB 2.6MB/s 


In [0]:
import logging
import sentencepiece as spm
import os

logging.basicConfig(level=logging.INFO)
input_file="to_mytxt2.txt"
vocab_size = 15000
prefix = 'm_bpe'


templates = '--input={} --model_prefix={} --vocab_size={} --pad_id=0 --unk_id=1 --bos_id=2  --eos_id=3 --model_type=bpe'
cmd = templates.format(input_file, prefix, vocab_size)

spm.SentencePieceTrainer.train(cmd)
print("BPE model trained!")


BPE model trained!


In [0]:
# makes segmenter instance and loads the model file (m.model)
sp_bpe = spm.SentencePieceProcessor()
sp_bpe2 = spm.SentencePieceProcessor()
sp_bpe.load('{}.model'.format(prefix))
sp_bpe2.load('{}.model'.format(prefix))
print("Trained BPE model loaded")


#option={"unk": "UNK","eos": "</s>"}
extra_options = 'bos:eos' #'reverse:bos:eos'
sp_bpe.SetEncodeExtraOptions(extra_options)

print("sp_bpe with extra options ")  
print(sp_bpe.EncodeAsIds('so good'))
print(sp_bpe.EncodeAsPieces('so good'))

#print("sp_bpe2 without extra options ")
#print(sp_bpe2.EncodeAsIds('so good'))
#print(sp_bpe2.EncodeAsPieces('so good'))

print(sp_bpe.DecodeIds([5]))
# vocabulary size
print(len(sp_bpe))

Trained BPE model loaded
sp_bpe with extra options 
[2, 76, 142, 3]
['<s>', '▁so', '▁good', '</s>']
a
15000


In [0]:
df.to_csv(r'/content/drive/My Drive/amazon-fine-food-reviews/to_mytxt2.txt', header=None, index=None, sep=' ', mode='a')


In [0]:
import warnings
warnings.filterwarnings("ignore")

df["Tokened_Summary"]=df["Summary"].apply(lambda x:sp_bpe.encode_as_pieces(x))
df["Tokened_Text"]=df["Text"].apply(lambda x:sp_bpe2.encode_as_pieces(x))
#val.head(10)


df.head()

,Summary,Text,Tokened_Summary,Tokened_Text
0,good quality dog food,i have bought several of the vitality canned d...,"[<s>, ▁good, ▁quality, ▁dog, ▁food, </s>]","[▁i, ▁have, ▁bought, ▁several, ▁of, ▁the, ▁vit..."
1,not as advertised,product arrived labeled as jumbo salted peanut...,"[<s>, ▁not, ▁as, ▁advertised, </s>]","[▁product, ▁arrived, ▁labeled, ▁as, ▁jumbo, ▁s..."
2,"""delight"" says it all",this is a confection that has been around a fe...,"[<s>, ▁"", delight, "", ▁says, ▁it, ▁all, </s>]","[▁this, ▁is, ▁a, ▁confection, ▁that, ▁has, ▁be..."
3,cough medicine,if you are looking for the secret ingredient i...,"[<s>, ▁cough, ▁medicine, </s>]","[▁if, ▁you, ▁are, ▁looking, ▁for, ▁the, ▁secre..."
4,great taffy,great taffy at a great price. there was a wid...,"[<s>, ▁great, ▁taffy, </s>]","[▁great, ▁taffy, ▁at, ▁a, ▁great, ▁price, ., ▁..."


In [0]:
df["Tokened_Summary"]=df["Summary"].apply(lambda x:sp_bpe.encode_as_ids(x))
df["Tokened_Text"]=df["Text"].apply(lambda x:sp_bpe2.encode_as_ids(x))
#val.head(10)
df.head()

,Summary,Text,Tokened_Summary,Tokened_Text
0,good quality dog food,i have bought several of the vitality canned d...,"[2, 142, 600, 215, 217, 3]","[7, 103, 502, 941, 46, 10, 14376, 1691, 215, 2..."
1,not as advertised,product arrived labeled as jumbo salted peanut...,"[2, 98, 115, 2886, 3]","[165, 1053, 4511, 115, 7604, 3440, 1844, 305, ..."
2,"""delight"" says it all",this is a confection that has been around a fe...,"[2, 34, 8147, 14969, 1365, 41, 176, 3]","[70, 49, 5, 8548, 85, 232, 378, 923, 5, 601, 4..."
3,cough medicine,if you are looking for the secret ingredient i...,"[2, 6793, 4362, 3]","[203, 78, 99, 760, 63, 10, 5191, 520, 50, 3119..."
4,great taffy,great taffy at a great price. there was a wid...,"[2, 172, 5947, 3]","[172, 5947, 167, 5, 172, 299, 14968, 313, 108,..."


In [0]:
type(df['Tokened_Summary'][0])

list

In [0]:
df['summSum']=df['Tokened_Summary'].str.len()
df['summSum'].describe()

count    568411.000000
mean          7.129556
std           3.335194
min           3.000000
25%           5.000000
50%           6.000000
75%           9.000000
max          92.000000
Name: summSum, dtype: float64

In [0]:
df['textSum']=df['Tokened_Text'].str.len()
df['textSum'].describe()

count    568411.000000
mean        100.644465
std         104.887953
min           3.000000
25%          41.000000
50%          69.000000
75%         121.000000
max        4803.000000
Name: textSum, dtype: float64

In [0]:
df.head()

,Summary,Text,Tokened_Summary,Tokened_Text,summSum,textSum
0,good quality dog food,i have bought several of the vitality canned d...,"[2, 142, 600, 215, 217, 3]","[7, 103, 502, 941, 46, 10, 14376, 1691, 215, 2...",6,52
1,not as advertised,product arrived labeled as jumbo salted peanut...,"[2, 98, 115, 2886, 3]","[165, 1053, 4511, 115, 7604, 3440, 1844, 305, ...",5,38
2,"""delight"" says it all",this is a confection that has been around a fe...,"[2, 34, 8147, 14969, 1365, 41, 176, 3]","[70, 49, 5, 8548, 85, 232, 378, 923, 5, 601, 4...",8,127
3,cough medicine,if you are looking for the secret ingredient i...,"[2, 6793, 4362, 3]","[203, 78, 99, 760, 63, 10, 5191, 520, 50, 3119...",4,49
4,great taffy,great taffy at a great price. there was a wid...,"[2, 172, 5947, 3]","[172, 5947, 167, 5, 172, 299, 14968, 313, 108,...",4,32


In [0]:
len(df)

568411

In [0]:
#https://stackoverflow.com/questions/18936957/count-distinct-words-from-a-pandas-data-frame
#df['Tokened_Text'].str.lower().str.split()

In [0]:
#remove columns which length of tokened_summary > 15 and length of tokened_text >60
df = df[(df['textSum'] <100)&(df['summSum']<9)].reset_index(drop=True)
len(df)

307463

In [0]:
df=df[['Tokened_Text','Tokened_Summary']]
df.head()

,Tokened_Text,Tokened_Summary
0,"[7, 103, 502, 941, 46, 10, 14376, 1691, 215, 2...","[2, 142, 600, 215, 217, 3]"
1,"[165, 1053, 4511, 115, 7604, 3440, 1844, 305, ...","[2, 98, 115, 2886, 3]"
2,"[203, 78, 99, 760, 63, 10, 5191, 520, 50, 3119...","[2, 6793, 4362, 3]"
3,"[172, 5947, 167, 5, 172, 299, 14968, 313, 108,...","[2, 172, 5947, 3]"
4,"[7, 542, 5, 3059, 2944, 63, 5947, 29, 674, 70,...","[2, 507, 5947, 3]"


In [0]:
len(df)

307463

In [0]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(df, test_size=0.3)
train.shape
#val.shape
train.head()

,Tokened_Text,Tokened_Summary
158891,"[88, 484, 154, 98, 1572, 32, 225, 70, 270, 295...","[2, 484, 219, 134, 14977, 3]"
15150,"[231, 46, 176, 46, 10, 630, 2822, 29, 422, 105...","[2, 184, 46, 741, 2018, 14977, 3]"
50918,"[70, 49, 10, 545, 165, 63, 445, 319, 3418, 149...","[2, 120, 14957, 14956, 20, 14961, 4425, 3]"
290189,"[802, 172, 1441, 63, 5, 215, 5155, 14972, 90, ...","[2, 552, 1537, 3]"
305477,"[41, 14973, 14953, 475, 14977, 29, 598, 701, 2...","[2, 475, 14977, 3]"


In [0]:
len(train)

215224

In [0]:
val.head()

,Tokened_Text,Tokened_Summary
95883,"[7, 3419, 70, 265, 223, 88, 926, 1273, 32, 292...","[2, 172, 265, 3]"
261352,"[10, 265, 232, 5, 1353, 14972, 1609, 157, 29, ...","[2, 342, 7, 1686, 3]"
214772,"[141, 99, 188, 188, 1136, 305, 41, 154, 103, 1...","[2, 2207, 5655, 5655, 888, 12850, 14977, 3]"
213935,"[141, 169, 185, 129, 10, 1011, 12655, 227, 7, ...","[2, 185, 115, 7, 2107, 14977, 3]"
247667,"[5, 357, 46, 2950, 1112, 141, 91, 2069, 171, 2...","[2, 2950, 3]"


In [0]:
len(val)

92239

In [0]:
with open('{}.vocab'.format(prefix), encoding='utf-8') as f:
    vocabs = [doc.strip() for doc in f]

print('num of vocabs = {}'.format(len(vocabs))) # 2000
print(sp_bpe.unk_id())
for id in range (4):
  print(sp_bpe.id_to_piece(id),sp_bpe.is_control(id))

num of vocabs = 15000
1
<pad> True
<unk> False
<s> True
</s> True


In [0]:
x_train=train.iloc[:,0]
x_val=val.iloc[:,0] 
y_train=train.iloc[:,1]
y_val=val.iloc[:,1]
x_train.head()

158891    [88, 484, 154, 98, 1572, 32, 225, 70, 270, 295...
15150     [231, 46, 176, 46, 10, 630, 2822, 29, 422, 105...
50918     [70, 49, 10, 545, 165, 63, 445, 319, 3418, 149...
290189    [802, 172, 1441, 63, 5, 215, 5155, 14972, 90, ...
305477    [41, 14973, 14953, 475, 14977, 29, 598, 701, 2...
Name: Tokened_Text, dtype: object

In [0]:
from keras.preprocessing.sequence import pad_sequences

#padding zero up to maximum length
x_train   =   pad_sequences(x_train,  maxlen=100, padding='post')
x_val   =   pad_sequences(x_val, maxlen=100, padding='post')
#padding zero up to maximum length
y_train    =   pad_sequences(y_train,  maxlen=9, padding='post')
y_val   =   pad_sequences(x_val, maxlen=9, padding='post')

print(x_train[14])

Using TensorFlow backend.


[    7  1294 14973 14949  2284  2106   400    63    10   299 14968   203
    85 14973 14953   342    78 14973     8  2284 14972   210    78   222
    61  1059 14968   715 14972    70   400    49   859   142    63    10
   299  1624 14968    41 14973 14953    67  2049    91     5  6839 14973
 14953   841 14968   172  4225 10061 14977     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0]


In [0]:
#we get total unique text tokens x_train =14698
x_voc=0
x_tokens=[]
for row in x_train:
  for tokenx in row:
    if tokenx not in x_tokens:
        x_tokens.append(tokenx)
        #print(token, end=' ')
        x_voc+=1

x_voc=x_voc+1
print(x_voc)


14515


In [0]:
x_voc_size=len(sp_bpe)
print(x_voc_size)

15000


In [0]:
#make a comparision between this one and next cell. 
#we get total unique text tokens y_train =10866
y_voc=0
y_tokens=[]

for row in y_train:
  for tokeny in row:
    if tokeny not in y_tokens:
      y_tokens.append(tokeny)
     
      y_voc+=1

y_voc=y_voc+1 

print(y_voc)

10220


In [0]:
y_train.shape

(215224, 9)

In [0]:
x_train.shape

(215224, 100)

In [0]:

#we get total unique text tokens y_train =15000 /actually this is the whole voc list size
y_voc2=0
y_tokens2=[]
t = 0
for row in y_train:
  import numpy as np
  t = max(t,np.max(row))
  for token in row:
    if token not in y_tokens2:
      y_tokens2.append(token)
      #print(token,end='')
      y_voc2+=1

y_voc2=y_voc+1 
y_voc2 = t+1

y_voc2_size=len(sp_bpe)#corrected from sp_bpe2 to sp_bpe
print(y_voc2_size)


15000


In [0]:
#import csv

#train_csv = train.to_csv (r'/content/drive/My Drive/amazon-fine-food-reviews/train-real.csv', index = None, header=True)
#val_csv = val.to_csv (r'/content/drive/My Drive/amazon-fine-food-reviews/val-real.csv', index = None, header=True)



In [0]:

print(len(train))
print(len(val))


215224
92239


In [0]:
#df = pd.read_csv(r'/content/drive/My Drive/amazon-fine-food-reviews/train-real.csv')
##Dataset is now stored in a Pandas Dataframe

#df_v = pd.read_csv(r'/content/drive/My Drive/amazon-fine-food-reviews/val-real.csv')
#len(df_v)

In [0]:
import tensorflow as tf

#from keras.preprocessing.text import Tokenizer 
#from nltk.corpus import stopwords   
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
from tensorflow.python.layers.core import Dense
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors
print('TensorFlow Version: {}'.format(tf.__version__))

TensorFlow Version: 1.15.0


In [0]:
import time
from tensorflow import keras
import datetime, os

#import datetime, os
from tensorflow.python.keras.callbacks import TensorBoard


In [0]:
from keras import backend as K
from tensorflow.keras.layers import *
from attention import *
K.clear_session()

latent_dim = 300
embedding_dim=100  # output vector from embedding layer

#encoder

max_text_len=100

encoder_inputs = Input(shape=(max_text_len,)) 
encoder_embedding = Embedding(x_voc_size, embedding_dim, trainable=True)(encoder_inputs)


#encoder_lstm1
encoder_LSTM1=LSTM(latent_dim, return_sequences=True, return_state=True, dropout=0.4, recurrent_dropout=0.4)
encoder_output1,state_h1,state_c1=encoder_LSTM1(encoder_embedding)

#encoder_lstm2
encoder_LSTM2=LSTM(latent_dim, return_sequences=True, return_state=True, dropout=0.4, recurrent_dropout=0.4)
encoder_output2,state_h2,state_c2=encoder_LSTM2(encoder_output1)

#encoder_lstm3
encoder_LSTM3=LSTM(latent_dim, return_sequences=True, return_state=True, dropout=0.4, recurrent_dropout=0.4)
encoder_outputs,state_h,state_c=encoder_LSTM3(encoder_output2)

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))

#embedding layer
dec_emb_layer = Embedding(y_voc2_size, embedding_dim,trainable=True)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True,dropout=0.4,recurrent_dropout=0.4)
decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c])

#Attention layer
attn_layer = AttentionLayer(name='attention_layer')
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])

# Concat attention input and decoder LSTM output
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

#dense layer
decoder_dense =  TimeDistributed(Dense(y_voc2_size, activation='softmax'))
decoder_outputs = decoder_dense(decoder_concat_input)

# Define the model 
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:numexpr.utils:NumExpr defaulting to 2 threads.


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 100)     1500000     input_1[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 100, 300), ( 481200      embedding[0][0]                  
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
______________________________________________________________________________________________

In [0]:
#Can I use TensorBoard with Google Colab?
#Tensorboard: ValueError: Duplicate plugins for name projector #22676
#https://github.com/pytorch/pytorch/issues/22676

logs_base_dir = "./logs"
logdir = os.path.join(logs_base_dir, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=8)

In [0]:
history=model.fit([x_train,y_train[:,:-1]], y_train.reshape(y_train.shape[0],y_train.shape[1], 1)[:,1:] ,epochs=5,callbacks=[es, tensorboard_callback],batch_size=384, validation_data=([x_val,y_val[:,:-1]], y_val.reshape(y_val.shape[0],y_val.shape[1], 1)[:,1:]))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 215985 samples, validate on 92566 samples
Epoch 1/5
215985/215985 [==============================] - 1149s 5ms/sample - loss: 2.7536 - val_loss: 0.2100
Epoch 2/5
215985/215985 [==============================] - 1135s 5ms/sample - loss: 2.3579 - val_loss: 0.2260
Epoch 3/5
215985/215985 [==============================] - 1139s 5ms/sample - loss: 2.1890 - val_loss: 0.2285
Epoch 4/5
215985/215985 [==============================] - 1141s 5ms/sample - loss: 2.0954 - val_loss: 0.2434
Epoch 5/5
215985/215985 [==============================] - 1133s 5ms/sample - loss: 2.0320 - val_loss: 0.2354


In [0]:
mytest = sp_bpe.encode_as_ids('love this bbq sauce... can\'t wait for this to be more available, i buy it up if i see a few at the local stores.')
newtest= sp_bpe.encode_as_ids('and I feel like a new man with this bag I looooove it !!')
print(mytest)
test = pad_sequences([newtest],  maxlen=100, padding='post')
print(test)
secNewTest=sp_bpe.DecodeIds(newtest)
#?? means UNK
print(secNewTest)

[2, 215, 66, 1696, 941, 313, 139, 14973, 14949, 1653, 62, 66, 31, 61, 226, 1111, 14972, 6, 254, 39, 248, 199, 6, 371, 5, 583, 165, 10, 771, 948, 14968, 3]
[[    2    30 14947     1   736   125     5   580  1154    90    66   287
  14947     1 10093  3028    39  8801     3     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0]]
and  ⁇  feel like a new man with this bag  ⁇  looooove it !!


In [0]:

# Encode the input sequence to get the feature vector
encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_outputs, state_h, state_c])

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input(shape=(max_text_len,latent_dim))

# Get the embeddings of the decoder sequence
dec_emb2= dec_emb_layer(decoder_inputs) 
# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

#attention inference
attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_inf_concat) 

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])

max_summary_len = 9
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    
    # Populate the first word of target sequence with the start word.
    target_seq[0, 0] = 2 #target_word_index['sostok']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
      
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        # Sample a token
        sampled_token_index = int(np.argmax(output_tokens[0, -1, :]))
        sampled_token = sp_bpe.DecodeIds([sampled_token_index]) #reverse_target_word_index[sampled_token_index]
        
        if(sampled_token!=3):#'eostok'):
            decoded_sentence += ' '+sampled_token

        # Exit condition: either hit max length or find stop word.
        if (sampled_token_index == 3 or sampled_token_index == 0 or len(decoded_sentence.split()) >= (max_summary_len-1)):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence
  
#decode_sequence(test)

In [0]:
#define the functions to convert an integer sequence to a word sequence for summary as well as the reviews:

def seq2summary(input_seq):
    newString=''
    for i in input_seq:
        if((i!=0 and i!=target_seq[2]) and i!=target_seq[3]):
            newString=newString+sp_bpe.DecodeIds[i]+' '
    return newString

def seq2text(input_seq):
    newString=''
    for i in input_seq:
        if(i!=0):
            newString=newString+sp_bpe2.DecodeIds[i]+' '
    return newString

In [0]:
!pip install rouge

In [0]:
#test some examples
from rouge import Rouge 

hypothesis = "the #### transcript is a written version of each day 's cnn student news program use this transcript to help students with reading comprehension and vocabulary use the weekly newsquiz to test your knowledge of storie s you     saw on cnn student news"

reference = "this page includes the show transcript use the transcript to help students with reading comprehension and vocabulary at the bottom of the page , comment for a chance to be mentioned on cnn student news . you must be a teac    her or a student age # # or older to request a mention on the cnn student news roll call . the weekly newsquiz tests     students ' knowledge of even ts in the news"

rouge = Rouge()
scores = rouge.get_scores(hypothesis, reference)
print(scores)

[{'rouge-1': {'f': 0.5238095189484127, 'p': 0.6285714285714286, 'r': 0.4489795918367347}, 'rouge-2': {'f': 0.2727272680991736, 'p': 0.375, 'r': 0.21428571428571427}, 'rouge-l': {'f': 0.4517704517703128, 'p': 0.5714285714285714, 'r': 0.40816326530612246}}]


In [0]:
for i in range(200,300):
    print("Review:",sp_bpe2.DecodeIds(x_train[i].tolist()))
    print("Original summary:",sp_bpe.DecodeIds(y_train[i].tolist()))
    print("Predicted summary:", decode_sequence(x_train[i].reshape(1,max_text_len)))
 
    #ref. https://pypi.org/project/rouge/
    rouge=Rouge()
    scores = rouge.get_scores(decode_sequence(x_train[i].reshape(1,max_text_len)),sp_bpe.DecodeIds(y_train[i].tolist()),avg=True)
    print(scores)
  
    print("\n")

Review: they just dont have that slim jim snap. really, they are kind of gross. m brother seems to like hem though so all is not lost.
Original summary: no snap
Predicted summary:  not so good 
{'rouge-1': {'f': 0.0, 'p': 0.0, 'r': 0.0}, 'rouge-2': {'f': 0.0, 'p': 0.0, 'r': 0.0}, 'rouge-l': {'f': 0.0, 'p': 0.0, 'r': 0.0}}


Review: there wonderful, my dog loves them, and i feel good giving them to him, i will always get these for him.
Original summary: my dog loves these!!!!!!!!!!!!!!!!!!
Predicted summary:  great product 
{'rouge-1': {'f': 0.0, 'p': 0.0, 'r': 0.0}, 'rouge-2': {'f': 0.0, 'p': 0.0, 'r': 0.0}, 'rouge-l': {'f': 0.0, 'p': 0.0, 'r': 0.0}}


Review: very unique and flavorful gummies. they don't have the texture or the chew of haribo, but they are very good in their own right. for the price you really can't go wrong.
Original summary: great for the price
Predicted summary:  great product 
{'rouge-1': {'f': 0.33333332888888895, 'p': 0.5, 'r': 0.25}, 'rouge-2': {'f': 0.0, 'p': 

In [0]:
%tensorboard --logdir logs
#click "Graphs" on the menu bar and take a look those generated grpahs

UsageError: Line magic function `%tensorboard` not found.


In [0]:

![This image is just show effect of tensorboard.](https://drive.google.com/file/d/1ak1cPo4oTgbYKMOkz0JxBw1QwQ9bJNIf/view?usp=sharing)